# Applications

## Tagging - POS

**Parts of speed tagging** - Gán nhãn loại từ:

- *POS Tagging* là quá trình gán nhãn (label) loại từ cho mỗi từ trong một câu, ví dụ như danh từ (N), động từ (V), tính từ (ADJ), trạng từ (ADV), giới từ (PREP), liên từ (CONJ), v.v.
- *Mục tiêu*: Từ một chuỗi các từ, dự đoán lớp từ tương ứng sao cho phản ánh đúng ngữ pháp và ngữ nghĩa trong ngôn ngữ đó.

**Ý nghĩa**

- Xác định loại từ là bước cơ bản và cần thiết trong nhiều ứng dụng NLP phức tạp hơn, ví dụ:
    - *Parsing*: Xây dựng cây phân tích cú pháp (syntactic parsing).
    - *Dịch máy*: Hỗ trợ xác định cấu trúc ngữ pháp, lựa chọn từ tương ứng chính xác.
    - *Trích xuất thông tin*: Hỗ trợ nhận diện và phân loại từ, cụm từ đặc biệt.
- POS Tagging giúp máy tính hiểu được chức năng ngữ pháp của từ trong câu, từ đó xử lý ngôn ngữ hiệu quả hơn.

**Phương pháp**

- *Thống kê cổ điển (HMM, CRF)*: Các mô hình ẩn Markov (HMM) và Mô hình trường ngẫu nhiên có điều kiện (CRF) dựa vào xác suất chuyển tiếp giữa các nhãn và xác suất quan sát từ để dự đoán nhãn POS.
- *Học sâu (Deep Learning)*: Các mô hình mạng nơ-ron (RNN, LSTM, BiLSTM, Transformers) kết hợp với embeddings (như Word2Vec, GloVe, hay contextual embeddings như BERT) cũng có thể được huấn luyện để gán nhãn POS một cách chính xác hơn, nhất là với những ngôn ngữ giàu ngữ cảnh như tiếng Việt.

In [2]:
import spacy
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt

data = pd.read_csv(
    r"contents\theory\aiml_algorithms\dl_nlp\data\94 - bbc-news.csv"
)
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Preprocessing
titles = data[["title"]].copy()

# lower case
titles["preprocessed"] = titles["title"].str.lower()

# stop word removal
en_stopwords = stopwords.words("english")
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: " ".join(
        [word for word in x.split() if word not in en_stopwords]
    )
)

# punctation removal
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: re.sub(r"[^\w\s]", "", x)
)

# word tokenization
titles["tokens_raw"] = titles["title"].apply(word_tokenize)
titles["tokens_preprocessed"] = titles["preprocessed"].apply(word_tokenize)

# lemmatizing
lemmatizer = WordNetLemmatizer()
titles["tokens_lemmatized"] = titles["tokens_preprocessed"].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in x]
)

# create lists for just our tokens
tokens_raw_list = sum(
    titles["tokens_raw"], []
)  # unpack our lists into a single list
tokens_clean_list = sum(titles["tokens_lemmatized"], [])

In [5]:
# create a spacy doc from our raw text - better for pos tagging
spacy_doc = nlp(" ".join(tokens_raw_list))

In [ ]:
# POS Tagging
pos = [{"token": token.text, "pos_tag": token.pos_} for token in spacy_doc]
pd.DataFrame(pos).head()

,token,pos_tag
0,Can,AUX
1,I,PRON
2,refuse,VERB
3,to,PART
4,work,VERB


## Tagging - NER

**Named Entity Recognition** – Nhận diện thực thể có tên

- *NER* là quá trình tự động phát hiện (dò tìm) và phân loại các thực thể “có tên” trong văn bản, ví dụ như tên người, tên địa điểm, tổ chức, thời gian, sự kiện, v.v.
- *Mục tiêu*: Xác định đâu là thực thể đặc biệt trong câu và chúng thuộc loại thực thể nào.
```text
Ví dụ câu tiếng Việt: “Ông Nguyễn Văn A làm việc tại Công ty Bkav ở Hà Nội.”
“Nguyễn Văn A” → Người (Person)
“Công ty Bkav” → Tổ chức (Organization)
“Hà Nội” → Địa điểm (Location)
```

**Ý nghĩa** 

NER là một bước quan trọng trong trích xuất thông tin:

- *Tìm kiếm và tổng hợp thông tin*: Cho phép hệ thống hiểu được đối tượng nào đang được đề cập đến (ví dụ, tìm tất cả các tin tức có nhắc đến một người/tổ chức cụ thể).
- *Hỏi đáp tự động (Question Answering)*: Giúp xác định câu trả lời chính xác khi câu hỏi hướng đến các thực thể (Ví dụ: “Ai là CEO của Google?”).
- *Phân tích ý kiến (Sentiment Analysis)*: Xem ý kiến của người dùng nhắm vào thực thể nào (nhãn hàng, địa điểm, cá nhân).

**Phương pháp**

- *Rule-based*: Sử dụng luật ngôn ngữ hoặc biểu thức chính quy (regex), kết hợp danh sách các thực thể đã biết. Cách này mang tính đặc thù, thiếu linh hoạt.
- *Thống kê/học máy cổ điển*: CRF, MaxEnt (Maximum Entropy), SVM… sử dụng đặc trưng (feature) về từ, ngữ cảnh, chữ hoa/thường, tiền tố/hậu tố, v.v.
- *Học sâu (Deep Learning)*:
    - Mô hình mạng nơ-ron nhiều tầng (BiLSTM + CRF) hoặc Transformers (BERT, XLM-R, PhoBERT…) huấn luyện đặc trưng tự động, thay vì thủ công.
    - Kết quả NER hiện nay cải thiện rất nhiều nhờ áp dụng mô hình Transformer, đặc biệt khi có dữ liệu gắn nhãn đủ lớn.

In [ ]:
ner = [
    {"token": token.text, "ner_tag": token.label_} for token in spacy_doc.ents
]
df = pd.DataFrame(ner).head()
df

,token,ner_tag
0,Liz Truss,PERSON
1,UK,GPE
2,Rationing,PRODUCT
3,superyachts,CARDINAL
4,Russian,NORP


In [ ]:
from spacy import displacy

displacy.render(spacy_doc[:150], style="ent", jupyter=True)

In [19]:
type(spacy_doc)

spacy.tokens.doc.Doc

## Sentiment Analysis

**Mục tiêu**: Nhằm gán nhãn (label) “tích cực”, “tiêu cực” hoặc “trung tính” cho một đoạn văn, câu, hoặc thậm chí ở mức độ chi tiết hơn (từng ý hoặc từ).

**Ứng dụng thực tế**:
- Phân tích phản hồi (review) sản phẩm, khách sạn, nhà hàng.
- Khai thác ý kiến trên mạng xã hội (Twitter, Facebook, v.v.).
- Xây dựng hệ thống hỗ trợ chăm sóc khách hàng tự động.
- Theo dõi và quản trị danh tiếng thương hiệu.

**Các phương pháp phổ biến trong Sentiment Analysis**

*1. Phương pháp dựa trên từ điển/ngữ nghĩa (Rule-based / Lexicon-based)*

- Từ điển cảm xúc (sentiment lexicon): Là bộ danh sách gồm các từ ngữ chứa hàm ý cảm xúc (từ tích cực như “tuyệt vời”, “đẹp”, “xuất sắc”, từ tiêu cực như “tệ”, “xấu”, “kinh khủng”,...). Mỗi từ thường gắn với một trọng số (score) phản ánh mức độ tích cực hoặc tiêu cực.
- Luật và quy tắc (rule): Áp dụng các quy tắc (ví dụ: nếu một câu có nhiều từ ngữ tiêu cực hơn thì khả năng cao câu đó mang nghĩa tiêu cực).

Ưu điểm:
- Dễ triển khai khi có sẵn từ điển cảm xúc.
- Giải thích được kết quả (biết từ nào quyết định cảm xúc).

Nhược điểm:
- Độ bao phủ (coverage) giới hạn nếu từ điển không đầy đủ.
- Khó nắm bắt bối cảnh (context) phức tạp, ví dụ mệnh đề phủ định hay các câu mỉa mai, châm biếm (sarcasm).
- Khó bảo trì và mở rộng từ điển theo ngôn ngữ, ngữ cảnh mới.

*2. Phương pháp Deep Learning Transformer*

- Transformer-based Models (BERT, RoBERTa, GPT, XLM-R, PhoBERT với tiếng Việt, v.v.)
- Transformer đã trở thành kiến trúc chuẩn cho nhiều bài toán NLP, bao gồm Sentiment Analysis.
- Các mô hình Pre-trained như BERT có khả năng biểu diễn ngôn ngữ tự nhiên rất tốt nhờ quá trình huấn luyện trên dữ liệu cực lớn.
- Khi fine-tuning cho bài toán phân tích cảm xúc, mô hình có thể đạt kết quả cao vượt trội.

Ưu điểm:
- Khả năng nắm bắt ngữ cảnh và mối quan hệ giữa các từ tốt hơn so với phương pháp cổ điển.
- Tính khái quát cao, mở rộng được cho các ngôn ngữ, miền dữ liệu khác nhau.

Nhược điểm:
- Đòi hỏi dữ liệu huấn luyện lớn, tài nguyên phần cứng (GPU/TPU) mạnh.
- Khó giải thích kết quả hơn so với phương pháp rule-based hay machine learning truyền thống.

**Quy trình triển khai một hệ thống Sentiment Analysis**
1. Thu thập dữ liệu: Lấy dữ liệu đánh giá, phản hồi, bình luận, v.v.
2. Tiền xử lý & Chuẩn hóa: Làm sạch, tách câu/từ, loại bỏ nhiễu, xử lý ký tự đặc biệt.
3. Chọn chiến lược tiếp cận:
- Rule-based / Lexicon-based: Cần có bộ từ điển cảm xúc phong phú, các quy tắc xử lý ngôn ngữ phù hợp.
- Machine Learning: Cần gán nhãn dữ liệu, trích xuất đặc trưng và huấn luyện.
- Deep Learning: Áp dụng mô hình embedding, RNN/CNN/Transformer, fine-tuning.
4. Huấn luyện và đánh giá:
- Chia dữ liệu thành các bộ train/validation/test.
- Đánh giá mô hình qua các chỉ số: Accuracy, Precision, Recall, F1-score.
5. Triển khai và giám sát:
- Đưa mô hình vào môi trường chạy thật.
- Thu thập phản hồi, đánh giá và tiếp tục cải thiện.

In [1]:
data = [
    {
        "text": "Sản phẩm này quá tuyệt vời, rất đáng đồng tiền.",
        "sentiment": "positive",
    },
    {
        "text": "Thật sự thất vọng, chất lượng kém hơn mong đợi.",
        "sentiment": "negative",
    },
    {
        "text": "Chưa biết xài sao, mới mua thôi, chắc ổn.",
        "sentiment": "neutral",
    },
    {
        "text": "Quá chán, mất thời gian, không đáng tiền chút nào.",
        "sentiment": "negative",
    },
    {
        "text": "Khá tốt, giá hợp lý, nhưng giao hàng hơi chậm.",
        "sentiment": "neutral",
    },
    {
        "text": "Thiết kế đẹp, màu sắc đúng ý, sẽ mua lại nếu cần.",
        "sentiment": "positive",
    },
    {
        "text": "Tuy sản phẩm không đẹp,  nhưng hài lòng chất lượng.",
        "sentiment": "positive",
    },
]

label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}


### Pre-trained model

In [ ]:
from transformers import pipeline

model_path = "5CD-AI/Vietnamese-Sentiment-visobert"
sentiment_pipeline = pipeline(
    "sentiment-analysis", model=model_path, tokenizer=model_path
)

for item in data:
    result = sentiment_pipeline(item["text"])[0]
    item["predicted_sentiment"] = result["label"]
    item["score"] = result["score"]

import pandas as pd

pd.DataFrame(data)

config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Device set to use cpu


,text,sentiment,predicted_sentiment,score
0,"Sản phẩm này quá tuyệt vời, rất đáng đồng tiền.",positive,POS,0.998884
1,"Thật sự thất vọng, chất lượng kém hơn mong đợi.",negative,NEG,0.998847
2,"Chưa biết xài sao, mới mua thôi, chắc ổn.",neutral,POS,0.659783
3,"Quá chán, mất thời gian, không đáng tiền chút ...",negative,NEG,0.999171
4,"Khá tốt, giá hợp lý, nhưng giao hàng hơi chậm.",neutral,POS,0.726378
5,"Thiết kế đẹp, màu sắc đúng ý, sẽ mua lại nếu cần.",positive,POS,0.993708
6,"Tuy sản phẩm không đẹp, nhưng hài lòng chất l...",positive,NEU,0.678071


### Custom model

train test split

In [ ]:
from sklearn.model_selection import train_test_split

# Chuyển data sang list
data_list = list(data)

# Bước 1: tách train (khoảng 60%) và tạm_thời (40%)
train_data, temp_data = train_test_split(
    data_list, test_size=0.4, random_state=42, shuffle=True
)

# Bước 2: từ temp_data tách ra validation (50%) và test (50%)
val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42, shuffle=True
)

print("Train size:", len(train_data))
print("Val size:", len(val_data))
print("Test size:", len(test_data))

Train size: 4
Val size: 1
Test size: 2


Tạo Dataset cho Hugging Face

In [ ]:
import torch
from torch.utils.data import Dataset


class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, label2id, max_len=64):
        """
        data: list of dict, mỗi dict có {"text": str, "sentiment": str}
        tokenizer: tokenizer từ Hugging Face (AutoTokenizer, v.v.)
        label2id: dict ánh xạ từ sentiment (str) -> id (int)
        max_len: chiều dài tối đa khi tokenize
        """
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        text = sample["text"]
        sentiment = sample["sentiment"]
        label_id = self.label2id[sentiment]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        # Mặc định, tokenizer trả về:
        #   encoding["input_ids"], encoding["attention_mask"], (có thể có token_type_ids)
        # Ta squeeze(0) để loại bỏ chiều batch (vì mỗi mẫu chỉ có 1)
        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        # Tạo dict kết quả
        item = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label_id, dtype=torch.long),
        }

        return item


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "5CD-AI/Vietnamese-Sentiment-visobert"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=3, id2label=id2label, label2id=label2id
)

train_dataset = SentimentDataset(train_data, tokenizer, label2id, max_len=64)
val_dataset = SentimentDataset(val_data, tokenizer, label2id, max_len=64)
test_dataset = SentimentDataset(test_data, tokenizer, label2id, max_len=64)

Metrics for evaluation

In [ ]:
import numpy as np


def evaluate_function(eval_pred):
    logits, labels = eval_pred  # logits, labels đều là numpy.ndarray
    # Dùng np.argmax thay vì torch.argmax
    predictions = np.argmax(logits, axis=-1)

    # Tính metric (VD: accuracy, f1...)
    # Ví dụ dùng scikit-learn hoặc huggingface metric
    # Ở đây minh hoạ load_metric("accuracy") và load_metric("f1")
    # (Bạn có thể thay bằng metric tuỳ ý)

    import evaluate

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(
        predictions=predictions, references=labels, average="weighted"
    )

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}


Training

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output/visobert-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./output/logs",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # dùng để validate
    processing_class=tokenizer,  # Trainer sẽ tự động batch & pad
    compute_metrics=evaluate_function,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.001200,5.580416,0.000000,0.000000
2,0.000300,8.432305,0.000000,0.000000
3,0.000500,8.990547,0.000000,0.000000


C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\transformers\configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 256}
  warnings.warn(
C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\transformers\configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This 

TrainOutput(global_step=6, training_loss=0.0007009292385191657, metrics={'train_runtime': 35.6154, 'train_samples_per_second': 0.337, 'train_steps_per_second': 0.168, 'total_flos': 394670126592.0, 'train_loss': 0.0007009292385191657, 'epoch': 3.0})

Evaluation

In [14]:
test_metrics = trainer.evaluate(test_dataset)
print("Test metrics:", test_metrics)

Test metrics: {'eval_loss': 3.8895528316497803, 'eval_accuracy': 0.5, 'eval_f1': 0.5, 'eval_runtime': 2.8813, 'eval_samples_per_second': 0.694, 'eval_steps_per_second': 0.347, 'epoch': 3.0}


In [ ]:
test_text = "Hàng giao nhanh, chất lượng vừa ý."
inputs = tokenizer(
    test_text, return_tensors="pt", truncation=True, padding=True
)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    pred_id = logits.argmax(dim=-1).item()

print("Text:", test_text)
print("Predicted sentiment:", id2label[pred_id])


Text: Hàng giao nhanh, chất lượng vừa ý.
Predicted sentiment: neutral


## Topic Modelling

**Topic modelling** là một nhánh trong Xử lý Ngôn ngữ Tự nhiên (NLP) nhằm **tìm ra các chủ đề tiềm ẩn (latent topics)** trong một tập văn bản lớn, đại diện cho 1 loại **unsupervised learning** trong NLP

**Ý tưởng cốt lõi là**: các văn bản thường có thể được mô tả bằng một số lượng hạn chế các chủ đề chính, và mỗi văn bản phản ánh các chủ đề đó với các trọng số (mức độ) khác nhau. Mục tiêu của topic modelling là tự động “phân loại” những văn bản này vào các chủ đề hoặc xác định được các chủ đề chính trong một tập dữ liệu lớn mà không cần gán nhãn trước.

--- 


**Khi nào nên sử dụng topic modelling?**

- *Khám phá dữ liệu (Exploratory Analysis)*: Khi bạn có một lượng văn bản lớn mà chưa biết rõ nội dung chính hay các nhóm chủ đề tiềm ẩn, topic modelling giúp bạn hình dung bức tranh tổng quan.
- *Phân nhóm tài liệu*: Khi bạn muốn phân nhóm các văn bản theo chủ đề để tiện cho quá trình quản lý, tìm kiếm, hoặc truy xuất thông tin.
    - VOC labeling
    - Social Listening   
- *Trích xuất từ khóa / insight*: Khi bạn muốn tóm tắt hoặc tìm các “từ khóa” nổi bật trong từng nhóm chủ đề nhằm phục vụ cho việc phân tích hoặc báo cáo.
- *Tiền xử lý cho các bài toán khác*: Thông tin về chủ đề có thể được sử dụng làm đặc trưng (features) cho các bài toán máy học khác, ví dụ: phân loại văn bản, khuyến nghị nội dung, hoặc phân tích cảm xúc.

---

**Các phương pháp phổ biến trong topic modelling**

- ***Latent Dirichlet Allocation (LDA):*** 

    Đây là phương pháp được sử dụng rất rộng rãi. Ý tưởng chính: giả định mỗi tài liệu (document) là tổ hợp của các chủ đề, và mỗi chủ đề là một phân phối xác suất trên các từ vựng. Thông qua quá trình huấn luyện, LDA ước lượng xem mỗi tài liệu chứa những chủ đề nào và mỗi chủ đề chứa những từ nào với trọng số bao nhiêu.
- ***Latent Semantic Analysis (LSA) / Latent Semantic Indexing (LSI):*** 

    Dựa trên kỹ thuật phân rã ma trận (thường là SVD). Mỗi tài liệu được biểu diễn dưới dạng vector “bag-of-words” (hoặc TF-IDF), sau đó giảm chiều để tìm “khoảng không gian ngữ nghĩa” ẩn. LSA/LSI cũng có thể được sử dụng để tìm các chủ đề tiềm ẩn, tuy nhiên thường không ràng buộc các chủ đề và từ theo kiểu phân phối xác suất như LDA.
- ***Non-negative Matrix Factorization (NMF):*** 

    Tương tự ý tưởng phân rã ma trận nhưng thêm ràng buộc “không âm” (non-negative). Vì vậy, các chủ đề NMF thường có diễn giải tương đối rõ ràng hơn (mỗi chủ đề là sự kết hợp cộng tuyến tính của các từ, không cho phép giá trị âm).
- ***Neural Topic Modelling (chủ đề dựa trên mạng nơ-ron):*** 

    Ví dụ: Neural Variational Document Model (NVDM) hay những mô hình deep learning khác. Chúng sử dụng mạng nơ-ron để học phân phối chủ đề ẩn trong tài liệu. Có tiềm năng thể hiện tốt với các dữ liệu lớn, phức tạp.
- ***BERT-based Topic Modelling:*** 

    Áp dụng các mô hình ngôn ngữ dựa trên Transformer như BERT để tạo embedding cho câu/từ. Sau đó, tiến hành clustering hoặc áp dụng các thuật toán phân nhóm khác lên embedding này để suy ra chủ đề.

---

**Cách xác định số lượng chủ đề (number of topics)**

Việc chọn số lượng chủ đề phù hợp rất quan trọng, vì số lượng chủ đề quá ít hoặc quá nhiều đều có thể làm giảm chất lượng phân tích. Một số phương pháp phổ biến để ước lượng “tối ưu”:

- ***Dựa trên kinh nghiệm / domain knowledge***

    - Nếu bạn đã biết trước lĩnh vực nội dung hoặc mang tính chuyên ngành, bạn có thể ước lượng số lượng chủ đề hợp lý dựa vào kinh nghiệm.
- ***Dùng metric đánh giá (Perplexity, Coherence, Silhouette Score, v.v.)***

    - **Coherence score** (C_v, UMass, v.v.) phổ biến trong đánh giá LDA. Ý tưởng: tính mức độ liên quan giữa các từ cùng chủ đề, hoặc giữa từ trong chủ đề với các tài liệu.
    - **Perplexity** thường được sử dụng cho các mô hình xác suất (vd. LDA), mặc dù nó không phải lúc nào cũng tương quan tốt với chất lượng “giải thích” của chủ đề.
    - **Elbow** method áp dụng cho một số kỹ thuật như NMF, spectral clustering, K-means (trong trường hợp clustering embedding).
    - **Grid search** (hoặc iterative search) với các mô hình topic modelling

- ***Huấn luyện nhiều mô hình với số lượng chủ đề khác nhau (k = 5, 10, 15, 20, …) và quan sát sự thay đổi của các chỉ số đánh giá (coherence, perplexity, …).***
    - Chọn k có chỉ số đánh giá tốt nhất hoặc đạt điểm cân bằng giữa giải thích được và tính gọn nhẹ.
    - Trong thực tế, kết hợp cả đánh giá định lượng và định tính (xem xét sự “giải thích được” của chủ đề) là cách tối ưu.

### Kết hợp mô hình truyền thống và transformer

**1. Vì sao phương pháp truyền thống (LDA, NMF…) vẫn còn giá trị?**

- Nhẹ, dễ triển khai: LDA, NMF triển khai tương đối gọn (thư viện Gensim, sklearn, …), không đòi hỏi hạ tầng GPU hay cấu hình quá cao.
- Tính giải thích (interpretability) rõ ràng:
    - Mỗi chủ đề là một phân phối xác suất của từ; dễ quan sát “top words” và gán nhãn.
    - Các tổ chức, doanh nghiệp trong lĩnh vực cần khả năng giải thích (compliance, pháp lý…) có thể ưu tiên LDA/NMF.
- Phù hợp dữ liệu vừa và nhỏ: Khi dữ liệu không quá lớn, hoặc chủ đề không quá phức tạp, LDA/NMF vẫn đem lại kết quả chấp nhận được, nhanh và ổn định.
- Đánh giá/ so sánh dễ dàng: Có sẵn các chỉ số như perplexity, coherence (C_v, UMass, …) và bộ công cụ phân tích.

---

**2. Lợi ích và hạn chế của tiếp cận transformer-based**

***Lợi ích***

- **Hiểu ngữ cảnh tốt hơn**: Các mô hình (BERT, PhoBERT, GPT, …) bắt được mối quan hệ ngữ nghĩa sâu hơn so với bag-of-words truyền thống.
- **Hiệu quả với ngôn ngữ phức tạp / nhiều ngôn ngữ**: Đặc biệt khi văn bản dài, từ vựng phong phú, hoặc nhiều biến thể.
- **Chất lượng chủ đề thường tốt hơn** (cụm từ, ngữ nghĩa phức tạp) nếu được huấn luyện/ fine-tune phù hợp.

***Hạn chế***

- **Chi phí tính toán cao**: Cần GPU, tài nguyên lớn hơn so với LDA/NMF.
- **Khó giải thích**: Transformer-based embedding đôi khi khó “truy ngược” được vì sao model nhóm văn bản đó vào một cụm; phải trích xuất top words thủ công thông qua các thống kê tần suất kết hợp (chứ không phải xác suất hiển thị rõ như LDA).
- **Phụ thuộc mô hình nền**: Chọn sai mô hình embedding, hoặc model chưa fine-tune cho domain → kết quả chưa tối ưu.

---

**3. Best Practice thực tế hiện nay**

- **Chọn phương pháp theo “bài toán” và “tài nguyên”:**

    - Nếu dữ liệu nhỏ hoặc vừa phải, yêu cầu giải thích cao, và tài nguyên hạn chế → LDA / NMF vẫn là lựa chọn hiệu quả.
    - Nếu dữ liệu lớn, mang tính đa dạng và cần nắm bắt ngữ nghĩa sâu, bạn sẵn sàng đầu tư hạ tầng → hướng transformer-based (BERTopic, Top2Vec, v.v.).

- **Kết hợp cả hai:**

    - Dùng embedding transformer để rút gọn chiều, sau đó chạy LDA trên vector (thay vì chạy LDA trên bag-of-words). Tuy không phổ biến bằng “transformer + clustering”, nhưng cũng là một hướng.

- **Bảo đảm pipeline “chuẩn”:**

    - Tiền xử lý: Với tiếng Việt, tách từ (Underthesea, PyVi…); với tiếng Anh, spaCy/NLTK. Lọc stopwords phù hợp.
    - Sinh embedding (nếu dùng transformer) hoặc TF-IDF (nếu LDA/NMF).
    - Giảm chiều (UMAP/PCA) → Phân cụm (K-means, HDBSCAN…).
    - Trích xuất top words để đặt tên chủ đề, đánh giá coherence.
    - Kiểm tra thủ công trên các văn bản đại diện.

- **Chú trọng đánh giá định tính (qualitative):**

    Dù dùng LDA hay BERT, vẫn nên xem xét thủ công một số văn bản trong cụm; vì chỉ số (coherence, silhouette…) cũng có thể chưa phản ánh hết.

- **Cân nhắc domain adaptation / fine-tuning (nếu dùng mô hình embedding):**

    - Đối với những ngữ cảnh chuyên ngành (y tế, tài chính, …), pretrained “chung” của BERT có thể không tối ưu.
    - Có thể fine-tune nhẹ (MLM, TSDAE, …) hoặc chọn mô hình domain-specific (BioBERT, FinBERT, PhoBERT news,…).

### Chuẩn bị và tiền xử lý dữ liệu

**1. Dọn dẹp, chuẩn hóa (cleaning, normalization):**

- Loại bỏ ký tự đặc biệt, link, HTML tag, …
- Xử lý chính tả, chuyển về chữ thường (nếu phù hợp).
- Tùy theo ngôn ngữ (tiếng Anh, tiếng Việt, …) sử dụng tokenizer phù hợp. Ví dụ:
    - Tiếng Anh: spaCy, NLTK, …
    - Tiếng Việt: Underthesea, PyVi, RDRsegmenter… (để tách từ).

**2. (Nếu cần) Tách câu hoặc chia nhỏ văn bản lớn:**

- Một bài báo quá dài có thể chia thành từng đoạn hoặc từng câu để embedding hiệu quả. Sau đó, có thể nhóm các đoạn/câu đó theo một cơ chế trung bình để đại diện toàn văn bản (hoặc áp dụng tiếp cận topic modelling trực tiếp trên từng đoạn/câu, rồi gộp lại).

**3. Loại bỏ stopwords/ từ ít thông tin:**

- Tùy ngôn ngữ, cần một danh sách stopwords chất lượng.
- Tuy nhiên, với tiếp cận embedding transformer, đôi khi những từ này không tác động tiêu cực nhiều như mô hình bag-of-words, nhưng vẫn nên cân nhắc lọc bớt “noise”.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"data\105 - news-articles.csv")
df.head()

,id,title,content
0,25626,"One Weight-Loss Approach Fits All? No, Not Eve...","Dr. Frank Sacks, a professor of nutrition at H..."
1,19551,South Carolina Stuns Baylor to Reach the Round...,South Carolina’s win over Duke was not only ...
2,25221,"U.S. Presidential Race, Apple, Gene Wilder: Yo...",(Want to get this briefing by email? Here’s th...
3,18026,"His Predecessor Gone, Gambia’s New President F...","BANJUL, Gambia — A week after he was inaugu..."
4,21063,‘Harry Potter and the Cursed Child’ Goes From ...,The biggest book of the summer isn’t a blockbu...


Preprocessing

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

# Gộp title + content nếu muốn (ở đây ví dụ gộp chung để phân tích toàn văn bản)
df["text"] = df["title"].astype(str) + " " + df["content"].astype(str)

# Tải stopwords nếu chưa có
nltk.download("stopwords")
nltk.download("punkt")

stop_words = set(stopwords.words("english"))
lmtz = WordNetLemmatizer()


def preprocess_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Loại bỏ dấu câu, từ rỗng và ký tự không cần thiết + lemmatize
    tokens = [
        lmtz.lemmatize(t)
        for t in tokens
        if t not in stop_words and t not in string.punctuation
    ]
    return tokens


# Áp dụng cho toàn bộ dữ liệu
df["tokens"] = df["text"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\datkt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\datkt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Sinh embedding từ mô hình Transformer

**2.1. Chọn mô hình gốc (pretrained model)**
- Tiếng Anh: BERT-base, RoBERTa-base, Sentence-BERT (SBERT), v.v.
- Tiếng Việt: PhoBERT, xlm-roberta-base (đa ngôn ngữ), hoặc các mô hình vibert4news,...
> Lưu ý:
>- Với dữ liệu chuyên ngành, nếu có domain-specific model (VD: mô hình cho y khoa, pháp luật, tài chính), bạn nên sử dụng để embedding phản ánh ngữ nghĩa chính xác hơn.
>- Có thể cân nhắc fine-tune nhẹ mô hình trên dữ liệu gốc (unsupervised domain adaptation) để cải thiện độ phù hợp.

**2.2. Chiến lược lấy embedding**
- **Sentence embedding:**

    - Ví dụ dùng Sentence-BERT (SBERT) – mô hình được huấn luyện đặc biệt để sinh embedding câu, cho kết quả tốt hơn so với BERT “raw”.
    - Nếu mô hình không được tối ưu cho sentence embedding, có thể dùng pooling (mean pooling, CLS token) từ layer cuối/ trung gian của BERT.

- **Document embedding:**

    - Nếu mỗi “bài” rất dài, có thể chia thành nhiều đoạn/câu, rồi lấy trung bình (average pooling), hoặc dùng mô hình hierarchical.

In [6]:
from sentence_transformers import SentenceTransformer

# Chọn 1 mô hình sentence-BERT cho tiếng Anh
model_name = 'all-MiniLM-L6-v2'
model_sbert = SentenceTransformer(model_name)

# Chuyển mỗi văn bản (đã preprocessing) thành chuỗi để embedding
docs = df['text'].tolist()

# Sinh embeddings (mỗi doc -> vector 384 chiều cho 'all-MiniLM-L6-v2')
embeddings = model_sbert.encode(docs, show_progress_bar=True)


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# (Tuỳ chọn) Giảm chiều để clustering tốt hơn: Dùng UMAP để giảm từ 384 chiều → 10, 15… (tuỳ ý).
import umap

reducer = umap.UMAP(n_neighbors=40, n_components=10, random_state=42)
reduced_embeddings = reducer.fit_transform(embeddings)

C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\datkt\Desktop\Working\notebooks\coding\learning\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


### Phân cụm

In [8]:
# K-means (cần biết trước số cụm):

from sklearn.cluster import KMeans

k = 5  # giả định muốn 5 cụm
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(reduced_embeddings)
cluster_labels = kmeans.labels_


In [ ]:
# HDBSCAN (tự động xác định số cụm dựa trên mật độ):

import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=10, gen_min_span_tree=True)
cluster_labels = clusterer.fit_predict(reduced_embeddings)


In [10]:
# Gán mỗi document vào cụm / chủ đề
df['cluster'] = cluster_labels
print(df[['id', 'title', 'cluster']].head())


      id                                              title  cluster
0  25626  One Weight-Loss Approach Fits All? No, Not Eve...        0
1  19551  South Carolina Stuns Baylor to Reach the Round...        4
2  25221  U.S. Presidential Race, Apple, Gene Wilder: Yo...        2
3  18026  His Predecessor Gone, Gambia’s New President F...        1
4  21063  ‘Harry Potter and the Cursed Child’ Goes From ...        3


### Trích xuất từ khoá đại diện cho từng cụm

- Cách nhanh: Đếm tần suất từ (hoặc TF-IDF) trong mỗi cụm.
- Hoặc: Sử dụng BERTopic – nó tự động thực hiện bước này.

In [11]:
# Trích xuất từ khoá đại diện cho từng cụm
from collections import Counter, defaultdict

cluster_texts = defaultdict(list)

for cluster_id, text_tokens in zip(df['cluster'], df['tokens']):
    cluster_texts[cluster_id].extend(text_tokens)

for c in sorted(cluster_texts.keys()):
    word_freq = Counter(cluster_texts[c])
    top_words = [w for w, _ in word_freq.most_common(10)]
    print(f"Cụm {c}: {top_words}")


Cụm 0: ['’', '“', '”', 'said', 'mr.', '—', 'would', 'dr.', 'one', 'company']
Cụm 1: ['’', '“', '”', 'said', 'mr.', 'state', '—', 'year', 'united', 'one']
Cụm 2: ['’', 'mr.', '“', '”', 'trump', 'said', '—', 'president', 'new', 'would']
Cụm 3: ['’', '“', '”', 'mr.', 'said', '—', 'new', 'time', 'show', 'song']
Cụm 4: ['’', '“', '”', 'said', 'new', 'mr.', 'ms.', '—', 'would', 'time']


- Nếu c == -1 (HDBSCAN): là outlier.
- Từ list top_words, bạn có thể đặt nhãn (ví dụ “Embeddings & BERT” / “LDA topic modeling”, …).

## Text classification

Text classification (phân loại văn bản) là một trong những nhiệm vụ quan trọng của xử lý ngôn ngữ tự nhiên (NLP), giúp gán nhãn cho văn bản dựa trên nội dung của nó. Ví dụ, phân loại email spam, phân loại tin tức theo chủ đề, phân loại đánh giá sản phẩm (tích cực/tiêu cực),…

### 1. Các bước trong pipeline phân loại văn bản

#### 1.1. Thu thập và chuẩn bị dữ liệu
- **Thu thập dữ liệu**: Tập hợp dữ liệu văn bản từ các nguồn như website, cơ sở dữ liệu, API,...
- **Gán nhãn**: Đối với bài toán giám sát, cần có dữ liệu đã được gán nhãn (ví dụ: spam vs. non-spam, tin tức thể thao, chính trị, giải trí, v.v.).
- **Tiền xử lý**:
  - Loại bỏ ký tự đặc biệt, HTML tags, số, dấu câu không cần thiết.
  - Chuyển đổi về chữ thường (lowercasing).
  - Tokenization: tách từ, câu bằng các công cụ như NLTK, spaCy (tiếng Anh) hoặc Underthesea (tiếng Việt).
  - Loại bỏ stopwords: loại bỏ các từ không mang nhiều ý nghĩa (ví dụ: "và", "nhưng", "của" trong tiếng Việt).
  - (Tuỳ chọn) Stemming/Lemmatization: rút gọn từ về gốc từ để giảm tính đa dạng của từ vựng.

#### 1.2. Biểu diễn văn bản (Text Representation)
- **Bag-of-Words (BoW)**: Biểu diễn văn bản dưới dạng vector tần số từ.
- **TF-IDF**: Cân nhắc tần số xuất hiện của từ trong văn bản so với toàn bộ corpus để xác định tầm quan trọng.
- **Word Embeddings**: Sử dụng các mô hình như Word2Vec, GloVe để chuyển từ thành vector có ngữ nghĩa.
- **Sentence/Document Embeddings**: Sử dụng các mô hình Transformer (BERT, RoBERTa, …) hoặc Sentence-BERT để chuyển đổi toàn bộ văn bản thành vector đại diện.

#### 1.3. Lựa chọn mô hình phân loại
- **Các thuật toán truyền thống**:
  - Naive Bayes
  - Support Vector Machines (SVM)
  - Logistic Regression
  - Decision Trees / Random Forest
- **Các mô hình deep learning**:
  - Convolutional Neural Networks (CNN) cho văn bản
  - Recurrent Neural Networks (RNN), LSTM, GRU
  - Transformer-based models (BERT, RoBERTa, XLNet, …) với fine-tuning

#### 1.4. Huấn luyện mô hình
- **Chia dữ liệu**: Tách dữ liệu thành tập huấn luyện, tập kiểm tra và (tuỳ chọn) tập validation.
- **Huấn luyện mô hình**: Sử dụng tập huấn luyện để học các đặc trưng và tìm ra các trọng số phù hợp.
- **Tối ưu hóa**: Sử dụng các thuật toán tối ưu (như Adam, SGD) và các kỹ thuật regularization để tránh overfitting.

#### 1.5. Đánh giá mô hình
- **Các chỉ số đánh giá**:
  - Accuracy: Độ chính xác tổng thể.
  - Precision, Recall, F1-score: Đặc biệt hữu ích với các lớp không cân bằng.
  - Confusion Matrix: Ma trận nhầm lẫn để phân tích lỗi.
- **Cross-validation**: Sử dụng K-fold cross-validation để đảm bảo mô hình ổn định và tổng quát.

#### 1.6. Triển khai và bảo trì
- **Triển khai mô hình**: Tích hợp vào hệ thống hoặc API để xử lý văn bản theo thời gian thực.
- **Bảo trì và cập nhật**: Theo dõi hiệu suất mô hình khi có dữ liệu mới và cập nhật mô hình định kỳ.

### 2. Các phương pháp hiện đại và ứng dụng

#### 2.1. Fine-tuning các mô hình Transformer
- **BERT và các biến thể**:
  - Fine-tuning BERT cho nhiệm vụ phân loại văn bản đã cho kết quả rất tốt, đặc biệt với dữ liệu có ngữ cảnh phức tạp.
  - Các mô hình như RoBERTa, XLNet, ALBERT cũng thường được sử dụng để cải thiện hiệu suất.
- **Ứng dụng**:
  - Phân loại email spam
  - Phân loại cảm xúc trong đánh giá sản phẩm
  - Phân loại tin tức, bài báo theo chủ đề

#### 2.2. Kết hợp các phương pháp truyền thống và deep learning
- **Hybrid models**:
  - Sử dụng các đặc trưng truyền thống (TF-IDF) kết hợp với embedding từ các mô hình Transformer.
  - Sử dụng ensemble methods: Kết hợp kết quả từ các mô hình truyền thống (Naive Bayes, SVM) với các mô hình deep learning để cải thiện độ chính xác.

### 3. Best practices trong Text Classification

- **Tiền xử lý**:
  - Làm sạch dữ liệu kỹ lưỡng là bước nền tảng.
  - Tùy chỉnh bộ stopwords và các bước tiền xử lý theo ngôn ngữ và đặc thù của dữ liệu.
- **Chọn đúng biểu diễn**:
  - Đối với văn bản ngắn, các biểu diễn như TF-IDF có thể hiệu quả.
  - Với văn bản phức tạp, sử dụng embeddings từ Transformer sẽ giúp nắm bắt ngữ cảnh tốt hơn.
- **Xây dựng pipeline linh hoạt**:
  - Sử dụng các thư viện như Scikit-learn, Keras, hoặc Hugging Face Transformers để xây dựng và huấn luyện mô hình.
  - Tích hợp các bước tiền xử lý, biểu diễn và huấn luyện thành một pipeline để dễ bảo trì.
- **Đánh giá kỹ càng**:
  - Sử dụng nhiều chỉ số đánh giá, đặc biệt khi dữ liệu có sự mất cân bằng giữa các lớp.
- **Fine-tuning và chuyển giao học**:
  - Fine-tuning mô hình pretrained trên tập dữ liệu của bạn sẽ cho hiệu quả cao.
  - Chuyển giao học (transfer learning) là chìa khóa để áp dụng thành công các mô hình Transformer trong NLP.

### 4. Kết luận

Text classification là một nhiệm vụ then chốt trong NLP với ứng dụng rộng rãi. Từ các phương pháp truyền thống như Naive Bayes, SVM đến các mô hình deep learning hiện đại như BERT, việc lựa chọn phương pháp phù hợp phụ thuộc vào đặc thù của dữ liệu, yêu cầu về hiệu suất và khả năng giải thích của mô hình. Một pipeline hoàn chỉnh cần kết hợp các bước từ thu thập, tiền xử lý, biểu diễn văn bản, huấn luyện mô hình đến đánh giá và triển khai, với khả năng điều chỉnh linh hoạt cho từng bài toán cụ thể.
